In [1]:
import pandas as pd
df = pd.read_csv("cyberbullying_tweets.csv")

In [2]:
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('omw')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Define pre-processing function
def preprocess_tweet(tweet):
    # Convert to lowercase
    tweet = tweet.lower()
    
    # Remove URLs
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Remove mentions and hashtags
    tweet = re.sub(r"@[^\s]+|#[^\s]+", "", tweet)
    
    # Remove non-alphabetic characters
    tweet = re.sub(r"[^a-zA-Z]", " ", tweet)
    
    # Tokenize the tweet
    tokens = nltk.word_tokenize(tweet)
    
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join the lemmatized tokens back into a string
    clean_tweet = " ".join(lemmatized_tokens)
    
    return clean_tweet

# Apply pre-processing to the "tweet_text" column and create a new column "clean_text"
df["clean_text"] = df["tweet_text"].apply(preprocess_tweet)

/Users/devansh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /Users/devansh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw to /Users/devansh/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/devansh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/devansh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/devansh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub



# Define BERT layer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=False)

# Define function to encode text using BERT layer
def get_bert_embeddings(texts):
    inputs = dict(
        input_ids=tf.keras.layers.Input(shape=(), dtype=tf.int32, name="input_word_ids"),
        input_mask=tf.keras.layers.Input(shape=(), dtype=tf.int32, name="input_mask"),
        segment_ids=tf.keras.layers.Input(shape=(), dtype=tf.int32, name="input_type_ids")
    )
    encoder = bert_layer(inputs)["pooled_output"]
    outputs = tf.keras.layers.Dense(768, activation="relu")(encoder)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    embeddings = model.predict(texts)
    return embeddings

# Encode text using BERT layer
embeddings = get_bert_embeddings(df["clean_text"])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df["cyberbullying_type"], test_size=0.2, random_state=42)

# Train Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, y_rf_pred)
print("Random Forest accuracy:", rf_acc)

# Train Logistic Regression model
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_lr_pred = lr.predict(X_test)
lr_acc = accuracy_score(y_test, y_lr_pred)
print("Logistic Regression accuracy:", lr_acc)
